In [2]:
import pandas as pd, numpy as np
import os, re

DESCRIPTION

Using NLP and ML, make a model to identify hate speech (racist or sexist tweets) in Twitter.

Problem Statement:  

Twitter is the biggest platform where anybody and everybody can have their views heard. Some of these voices spread hate and negativity. Twitter is wary of its platform being used as a medium  to spread hate. 

You are a data scientist at Twitter, and you will help Twitter in identifying the tweets with hate speech and removing them from the platform. You will use NLP techniques, perform specific cleanup for tweets data, and make a robust model.

Domain: Social Media

Analysis to be done: Clean up tweets and build a classification model by using NLP techniques, cleanup specific for tweets data, regularization and hyperparameter tuning using stratified k-fold and cross-validation to get the best model.

Content: 

id: identifier number of the tweet

Label: 0 (non-hate) /1 (hate)

Tweet: the text in the tweet

Tasks: 

1> Load the tweets file using read_csv function from Pandas package. 

2> Get the tweets into a list for easy text cleanup and manipulation.

3> To cleanup: 

    - Normalize the casing.

    - Using regular expressions, remove user handles. These begin with '@’.

    - Using regular expressions, remove URLs.

    - Using TweetTokenizer from NLTK, tokenize the tweets into individual terms.

    - Remove stop words.

    - Remove redundant terms like ‘amp’, ‘rt’, etc.

    - Remove ‘#’ symbols from the tweet while retaining the term.

4> Extra cleanup by removing terms with a length of 1.

5> Check out the top terms in the tweets:

    - First, get all the tokenized terms into one large list.

    - Use the counter and find the 10 most common terms.

6> Data formatting for predictive modeling:

    - Join the tokens back to form strings. This will be required for the vectorizers.

    - Assign x and y.

    - Perform train_test_split using sklearn.

7> We’ll use TF-IDF values for the terms as a feature to get into a vector space model.

    - Import TF-IDF  vectorizer from sklearn.

    - Instantiate with a maximum of 5000 terms in your vocabulary.

    - Fit and apply on the train set.

    - Apply on the test set.

8> Model building: Ordinary Logistic Regression

    - Instantiate Logistic Regression from sklearn with default parameters.

    - Fit into  the train data.

    - Make predictions for the train and the test set.

9> Model evaluation: Accuracy, recall, and f_1 score.

    - Report the accuracy on the train set.

    - Report the recall on the train set: decent, high, or low.

    - Get the f1 score on the train set.

10> Looks like you need to adjust the class imbalance, as the model seems to focus on the 0s.

    - Adjust the appropriate class in the LogisticRegression model.

11> Train again with the adjustment and evaluate.

    - Train the model on the train set.

    - Evaluate the predictions on the train set: accuracy, recall, and f_1 score.

12> Regularization and Hyperparameter tuning:

    - Import GridSearch and StratifiedKFold because of class imbalance.

    - Provide the parameter grid to choose for ‘C’ and ‘penalty’ parameters.

    - Use a balanced class weight while instantiating the logistic regression.

13> Find the parameters with the best recall in cross validation.

    - Choose ‘recall’ as the metric for scoring.

    - Choose stratified 4 fold cross validation scheme.

    - Fit into  the train set.

14> What are the best parameters?

15> Predict and evaluate using the best estimator.

    - Use the best estimator from the grid search to make predictions on the test set.

    - What is the recall on the test set for the toxic comments?

    - What is the f_1 score?

#### Read in the csv using pandas 

In [3]:
inp_tweets0 = pd.read_csv("C:/Users/sharv/Documents/Post Graduate Program In Data Science-PurdueSimpliLearn/Course4-NLP/Projects/TwitterHate.csv")
inp_tweets0.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
inp_tweets0.label.value_counts(normalize=True)

0    0.929854
1    0.070146
Name: label, dtype: float64

In [5]:
inp_tweets0.tweet.sample().values[0]

'sea shepherd suppoers are racist!   #antiracism #seashepherd  '

#### Get the tweets into a list, for easy text clean up and manipulation

In [6]:
tweets0 = inp_tweets0.tweet.values

In [7]:
len(tweets0)

31962

In [8]:
tweets0[:5]

array([' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
       "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
       '  bihday your majesty',
       '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
       ' factsguide: society now    #motivation'], dtype=object)

The tweets contain - 
1. URLs
2. Hashtags
3. User handles
4. 'RT'

## Cleanup 

#### Normalizing case

In [9]:
tweets_lower = [twt.lower() for twt in tweets0]

In [10]:
tweets_lower[:5]

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

#### Remove user handles, begin with '@'

In [11]:
import re

In [12]:
re.sub("@\w+","", "@Rahim this course rocks! http://rahimbaig.com/ai")

' this course rocks! http://rahimbaig.com/ai'

In [13]:
tweets_nouser = [re.sub("@\w+","", twt) for twt in tweets_lower]

In [14]:
tweets_nouser[:5]

['  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

#### Remove URLs

In [15]:
re.sub("\w+://\S+","", "@Rahim this course rocks! http://rahimbaig.com/ai")

'@Rahim this course rocks! '

In [16]:
tweets_nourl = [re.sub("\w+://\S+","", twt) for twt in tweets_nouser]

In [17]:
tweets_nourl[:5]

['  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

#### Tokenze using Tweet Tokenizer from NLTK

In [18]:
from nltk.tokenize import TweetTokenizer

In [19]:
?TweetTokenizer()

Object `TweetTokenizer()` not found.


In [20]:
tkn = TweetTokenizer()

In [21]:
print(tkn.tokenize(tweets_nourl[0]))

['when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', '.', '#run']


In [22]:
tweet_token = [tkn.tokenize(sent) for sent in tweets_nourl]
print(tweet_token[0])

['when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', '.', '#run']


### Remove punctuations and stop words and other redundant terms tike 'rt', 'amp'
- Also remove hashtags

In [23]:
from nltk.corpus import stopwords
from string import punctuation

In [24]:
stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)

In [25]:
stop_punct.extend(['...','``',"''",".."])

In [26]:
stop_context = ['rt', 'amp']

In [27]:
stop_final = stop_nltk + stop_punct + stop_context

#### Function to 
- remove stop words from a single tokenized sentence
- remove # tags
- remove terms with length = 1

In [28]:
def del_stop(sent):
    return [re.sub("#","",term) for term in sent if ((term not in stop_final) & (len(term)>1))]

In [29]:
del_stop(tweet_token[4])

['factsguide', 'society', 'motivation']

In [30]:
tweets_clean = [del_stop(tweet) for tweet in tweet_token]

#### Check out the top terms in the tweets

In [31]:
from collections import Counter

In [32]:
term_list = []
for tweet in tweets_clean:
    term_list.extend(tweet)

In [33]:
res = Counter(term_list)
res.most_common(10)

[('love', 2748),
 ('day', 2276),
 ('happy', 1684),
 ('time', 1131),
 ('life', 1118),
 ('like', 1047),
 ("i'm", 1018),
 ('today', 1013),
 ('new', 994),
 ('thankful', 946)]

## Data formatting for predictive modeling 

#### Join the tokens back into strings

In [34]:
tweets_clean[0]

['father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction', 'run']

In [35]:
tweets_clean = [" ".join(tweet) for tweet in tweets_clean]

In [36]:
tweets_clean[0]

'father dysfunctional selfish drags kids dysfunction run'

### Separate X and Y and perform train test split, 70-30

In [37]:
len(tweets_clean)

31962

In [38]:
len(inp_tweets0.label)

31962

In [39]:
X = tweets_clean
y = inp_tweets0.label.values

####  Train test split

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

### Create a document term matrix using count vectorizer

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vectorizer = TfidfVectorizer(max_features = 5000)

In [43]:
len(X_train), len(X_test)

(22373, 9589)

In [44]:
X_train_bow = vectorizer.fit_transform(X_train)

X_test_bow = vectorizer.transform(X_test)

In [45]:
X_train_bow.shape, X_test_bow.shape

((22373, 5000), (9589, 5000))

### Model building

### Using a *simple* Logistic Regression

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
logreg = LogisticRegression()

In [48]:
logreg.fit(X_train_bow, y_train)
print(logreg.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [49]:
y_train_pred = logreg.predict(X_train_bow)
y_test_pred = logreg.predict(X_test_bow)

In [50]:
from sklearn.metrics import accuracy_score, classification_report

In [51]:
accuracy_score(y_train, y_train_pred)

0.9560184150538595

In [52]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     20815
           1       0.96      0.39      0.55      1558

    accuracy                           0.96     22373
   macro avg       0.96      0.69      0.76     22373
weighted avg       0.96      0.96      0.95     22373



#### Adjusting for class imbalance

In [53]:
logreg = LogisticRegression(class_weight="balanced")

In [54]:
logreg.fit(X_train_bow, y_train)
print(logreg.get_params())

{'C': 1.0, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [55]:
y_train_pred = logreg.predict(X_train_bow)
y_test_pred = logreg.predict(X_test_bow)

In [56]:
accuracy_score(y_train, y_train_pred)

0.9527108568363652

In [57]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97     20815
           1       0.60      0.97      0.74      1558

    accuracy                           0.95     22373
   macro avg       0.80      0.96      0.86     22373
weighted avg       0.97      0.95      0.96     22373



In [58]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [59]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'penalty': ["l1","l2"]
}

In [64]:
?LogisticRegression()

Object `LogisticRegression()` not found.


In [60]:
classifier_lr = LogisticRegression(class_weight="balanced")

In [61]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = classifier_lr, param_grid = param_grid, 
                          cv = StratifiedKFold(4), n_jobs = -1, verbose = 1, scoring = "recall" )

In [62]:
grid_search.fit(X_train_bow, y_train)
print(grid_search.get_params())

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    7.5s finished


{'cv': StratifiedKFold(n_splits=4, random_state=None, shuffle=False), 'error_score': nan, 'estimator__C': 1.0, 'estimator__class_weight': 'balanced', 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__l1_ratio': None, 'estimator__max_iter': 100, 'estimator__multi_class': 'auto', 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'lbfgs', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(class_weight='balanced'), 'iid': 'deprecated', 'n_jobs': -1, 'param_grid': {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': 'recall', 'verbose': 1}


In [63]:
grid_search.best_estimator_

LogisticRegression(C=1, class_weight='balanced')

### Using the best estimator to make predictions on the test set

In [64]:
y_test_pred = grid_search.best_estimator_.predict(X_test_bow)

In [65]:
y_train_pred = grid_search.best_estimator_.predict(X_train_bow)

In [66]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      8905
           1       0.49      0.77      0.60       684

    accuracy                           0.93      9589
   macro avg       0.73      0.85      0.78      9589
weighted avg       0.95      0.93      0.93      9589

